In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pathlib
import os

# In[50]:

from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras import models
from PIL import Image

In [2]:
# 获取 数据目录
# 数据集用 https://www.kaggle.com/moltean/fruits/data 
data_root_orig = "/root/quan/fruits-360/Training/"
data_root = pathlib.Path(data_root_orig)
label_name = sorted(item.name for item in data_root.glob('*/')
                    if item.is_dir())
# label 名称 
label_name

/root/quan/fruits-360/Training
/root/quan/fruits-360/Test


['Apple Braeburn',
 'Apple Crimson Snow',
 'Apple Golden 1',
 'Apple Golden 2',
 'Apple Golden 3',
 'Apple Granny Smith',
 'Apple Pink Lady',
 'Apple Red 1',
 'Apple Red 2',
 'Apple Red 3',
 'Apple Red Delicious',
 'Apple Red Yellow 1',
 'Apple Red Yellow 2',
 'Apricot',
 'Avocado',
 'Avocado ripe',
 'Banana',
 'Banana Lady Finger',
 'Banana Red',
 'Beetroot',
 'Blueberry',
 'Cactus fruit',
 'Cantaloupe 1',
 'Cantaloupe 2',
 'Carambula',
 'Cauliflower',
 'Cherry 1',
 'Cherry 2',
 'Cherry Rainier',
 'Cherry Wax Black',
 'Cherry Wax Red',
 'Cherry Wax Yellow',
 'Chestnut',
 'Clementine',
 'Cocos',
 'Dates',
 'Eggplant',
 'Ginger Root',
 'Granadilla',
 'Grape Blue',
 'Grape Pink',
 'Grape White',
 'Grape White 2',
 'Grape White 3',
 'Grape White 4',
 'Grapefruit Pink',
 'Grapefruit White',
 'Guava',
 'Hazelnut',
 'Huckleberry',
 'Kaki',
 'Kiwi',
 'Kohlrabi',
 'Kumquats',
 'Lemon',
 'Lemon Meyer',
 'Limes',
 'Lychee',
 'Mandarine',
 'Mango',
 'Mango Red',
 'Mangostan',
 'Maracuja',
 'Melon

In [3]:

# the tfrecord file path, you need to create the folder yourself
# 生成tf record 目录
recordPath = "./tfrecord/"
if not os.path.exists(recordPath):
    os.makedirs(recordPath)

# the number of classes of images

# 组装 label名称和标签名称 例如 {“name”：1}
values = [i for i in list(range(len(label_name)))]
classes = dict(zip(label_name, values))


In [4]:
# 创建数据格式
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def image_example(image_string, label_name):
      image_shape = tf.image.decode_jpeg(image_string).shape

      feature = {
          'label': _int64_feature(label_name),

          'image_raw': _bytes_feature(image_string),
      }
      return tf.train.Example(features=tf.train.Features(feature=feature))
    

In [5]:
#写入文件 
for name, label in classes.items():
    class_path = os.path.join(data_root_orig, name)
    tf_file_name = "train.tfrecords-%.3d" % label
    with tf.io.TFRecordWriter(tf_file_name) as writer:
        for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                img = open(img_path, "rb").read()
                example = image_example(img, label)
                writer.write(example.SerializeToString())


In [6]:
# 文件类别
image_feature_description = {

    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}
# 提供组装 数据方法
def _parse_image_function(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

In [297]:
# 加载tfrecord 文件 
os.chdir("/root/quan/fruits-360/tfrecord/tfrecord/")
print(os.getcwd())
name_list = [k for _, _ , k in os.walk(".")]
print(name_list[0])
dataset = tf.data.TFRecordDataset(filenames = [name_list[0]])
dataset

/root/quan/fruits-360/tfrecord/tfrecord
['train.tfrecords-103', 'train.tfrecords-003', 'train.tfrecords-076', 'train.tfrecords-004', 'train.tfrecords-070', 'train.tfrecords-008', 'train.tfrecords-030', 'train.tfrecords-026', 'train.tfrecords-052', 'train.tfrecords-048', 'train.tfrecords-011', 'train.tfrecords-083', 'train.tfrecords-014', 'train.tfrecords-027', 'train.tfrecords-016', 'train.tfrecords-021', 'train.tfrecords-101', 'train.tfrecords-079', 'train.tfrecords-098', 'train.tfrecords-092', 'train.tfrecords-000', 'train.tfrecords-117', 'train.tfrecords-067', 'train.tfrecords-009', 'train.tfrecords-074', 'train.tfrecords-080', 'train.tfrecords-110', 'train.tfrecords-033', 'train.tfrecords-072', 'train.tfrecords-041', 'train.tfrecords-049', 'train.tfrecords-005', 'train.tfrecords-024', 'train.tfrecords-115', 'train.tfrecords-081', 'train.tfrecords-002', 'train.tfrecords-032', 'train.tfrecords-051', 'train.tfrecords-111', 'train.tfrecords-062', 'train.tfrecords-088', 'train.tfrecords

<TFRecordDatasetV2 shapes: (), types: tf.string>

In [312]:
parsed_image_dataset = dataset.map(_parse_image_function)
# for image_feature in parsed_image_dataset:
#     #print(image_feature)
#     img_raw = image_feature['label'].numpy()
#     print(img_raw)


In [316]:
datas =  parsed_image_dataset.shuffle(buffer_size=10).batch(50)
